### Calculate deaths from PM2.5

In [1]:
import geopandas as gpd
import xarray as xr
from cartopy import crs as ccrs
import seaborn as sns; sns.set_theme()
import os
import fiona
import country_converter as coco
import dask
import dask.array as da
import netCDF4 as nc
import regionmask
from matplotlib import cm
import numpy as np
from matplotlib import pyplot as plt
import country_converter as coco
import pyogrio
#pyogrio.set_gdal_config_options({"SHAPE_RESTORE_SHX": "YES"})
import pandas as pd
from cartopy.util import add_cyclic_point
import nc_time_axis
import glob
import cdo
import pandas as pd
import cartopy.feature as cfeature


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Gridded Population of the World, Version 4 (GPWv4): National Identifier Grid, Revision 11
# can be downloaded from: https://www.earthdata.nasa.gov/data/catalog/sedac-ciesin-sedac-gpwv4-natiden-r11-4.11#:~:text=Description,use%20in%20aggregating%20population%20data.
# cited as: Center For International Earth Science Information Network-CIESIN-Columbia University. (2018). Gridded Population of the World, Version 4 (GPWv4): National Identifier Grid, Revision 11 (Version 4.11) [Data set]. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/H4TD9VDP
shapefile_path = "./gpw-v4-national-identifier-grid-rev11_15_min_asc/gpw_v4_national_identifier_grid_rev11_15_min.shp"

# Read the shapefile
gdf = gpd.read_file(shapefile_path)
#gdf_list = pd.read_csv(f'./Mortality_data/country_borders/gpw_new.csv')

In [5]:
#mortality = xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/BMR/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
mortality_cardio_female = xr.open_mfdataset(f'./bmr/Cardio/female/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
mortality_cardio_male = xr.open_mfdataset(f'./bmr/Cardio/male/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']

mortality_resp_female = xr.open_mfdataset(f'./bmr/Resp/female/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
mortality_resp_male = xr.open_mfdataset(f'./bmr/Resp/male/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']

#mortality_cardio = xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/BMR/for_o3/cardio/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']
#mortality_resp = xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/BMR/for_o3/resp/*.nc', parallel=True, combine='nested', concat_dim='time')['mortality']

#mortality_og = pd.read_csv(f'./Mortality_data/IFs/mortality_all_new.csv')

#mortality per thousand
demographic_female = xr.open_mfdataset(f'./demographic/countries/female_2020_demo_frac.nc')['demo_pop']
demographic_male = xr.open_mfdataset(f'./demographic/countries/male_2020_demo_frac.nc')['demo_pop']

#demographic = demographic.sum("age")

#used to make a mask
population = xr.open_dataset(f'./population/SSP2/Total/NetCDF/ssp2_2020.nc')


In [6]:
#get country mask
lon = population['lon']
lat = population['lat']
print(lat.shape)
mask = regionmask.mask_3D_geopandas(gdf,lon,lat)

(1117,)


In [18]:
mortality_cardio['time'] = AF_cardio_0['time'].values
mortality_resp['time'] = AF_cardio_0['time'].values

### Calculating health impacts associated with air pollution (PM2.5)

Calculate mortalities attributed to PM2.5 from noncommunicable diseases and lower respiratory infections (combined) based on epidemiological cohort studies of long-term exposure to PM2.5

\begin{equation}
M = BMR \times P \times AF
\end{equation}

- M = mortality due to PM2.5
- BMR = baseline mortality rate
- P = exposed population
- AF = attributable factor

In [7]:
mortality_cardio = ((mortality_cardio_female* (demographic_female.values/1e3))+(mortality_cardio_male* (demographic_male.values/1e3)))
mortality_resp = ((mortality_resp_female* (demographic_female.values/1e3))+(mortality_resp_male* (demographic_male.values/1e3)))

mortality_cardio = mortality_cardio.sum("age")
mortality_resp = mortality_resp.sum("age")

In [47]:
ensemble = '010'#'010'

AF_cardio = -xr.open_mfdataset(f'./RR_ozone/arise-sai-1.0/{ensemble}.nc')['RR_card_em'] + xr.open_mfdataset(f'./RR_ozone/ssp245/{ensemble}.nc')['RR_card_base'].mean("time")
AF_resp = -xr.open_mfdataset(f'./RR_ozone/arise-sai-1.0/{ensemble}.nc')['RR_resp_em'] + xr.open_mfdataset(f'./RR_ozone/ssp245/{ensemble}.nc')['RR_resp_base'].mean("time")

#AF_cardio = -xr.open_mfdataset(f'./Volumes/bkup_srm/SRM/RR_ozone/{ensemble}.nc')['RR_card_base'] + xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR_ozone/ssp245/{ensemble}.nc')['RR_card_base'].mean("time")
#AF_resp = -xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR_ozone/{ensemble}.nc')['RR_resp_base'] + xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR_ozone/ssp245/{ensemble}.nc')['RR_resp_base'].mean("time")

#AF_cardio = 1-xr.open_mfdataset(f'./RR_ozone/arise-sai-1.0/{ensemble}.nc')['RR_card_base']#.mean("time")# + xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/RR_ozone/{ensemble}.nc')['RR_card_base']#[0]
#AF_resp = 1-xr.open_mfdataset(f'./RR_ozone/ssp245/{ensemble}.nc')['RR_resp_base']#.mean("time")# + xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/RR_ozone/{ensemble}.nc')['RR_resp_base'][0]


In [48]:
deaths = (AF_cardio* mortality_cardio[5:]) + (AF_resp* mortality_resp[5:])
#AF  = (AF_cardio*mortality_cardio[:10])#+ (AF_resp*mortality_resp[:10])
#AF  = (AF_cardio*mortality_cardio[:10].mean("time"))+(AF_resp*mortality_resp[:10].mean("time"))



In [49]:
deaths_val = deaths.where(deaths != 0, np.nan)


In [50]:
#ensemble = 'deaths_ensemble'
deaths_val.to_netcdf(f'./deaths_ozone/arise-sai-1.0/by_grid/{ensemble}.nc')


In [41]:
num_ensembles = 10
for i in range(1, num_ensembles + 1):
    ensemble = f'{i:03d}' 
    AF_cardio = 1-xr.open_mfdataset(f'/Volumes/bkup_srm/SRM/RR_ozone/{ensemble}.nc')['RR_card_base']#.mean("time")# + xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/RR_ozone/{ensemble}.nc')['RR_card_base']#[0]
    AF_resp = 1-xr.open_mfdataset(f'/Volumes/bkup_srm/SRM//RR_ozone/{ensemble}.nc')['RR_resp_base']#.mean("time")# + xr.open_mfdataset(f'/glade/derecho/scratch/cindywang625/RR_ozone/{ensemble}.nc')['RR_resp_base'][0]
    deaths = (AF_cardio* mortality_cardio[5:]) + (AF_resp* mortality_resp[5:])

    deaths_val = deaths.where(deaths != 0, np.nan)
    deaths_val.to_netcdf(f'/Volumes/bkup_srm/SRM/deaths_ozone/ssp245/by_grid/{ensemble}.nc')

    